In [19]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import numpy as np

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [85]:
def get_hyponyms(word):
    word = wn.synsets(word)
    if len(word) == 0:
        return []
    word = word[0]
    hyponyms = list(set([w for s in word.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    
    return hyponyms

In [86]:
def get_hypernyms(word):
    word = wn.synsets(word)
    if len(word) == 0:
        return []
    word = word[0]
    hypernyms = list(set([w for s in word.closure(lambda s:s.hypernyms()) for w in s.lemma_names()]))
    
    return hypernyms

In [87]:
def get_synonyms(word):
    word = wn.synsets(word)
    if len(word) == 0:
        return []
    synonyms = list(set([w for s in word for w in s.lemma_names()]))
    
    return synonyms

In [91]:
get_synonyms('apple_tree')

['apple_tree']

In [34]:
def get_vector(word1, word2):
    
    
    w1 = np.zeros(3)
    w2 = np.zeros(3)
    
    # check hyper/hypo-nym
    if w2n:
        if w1n in w2n.hypernyms():
            w1[0] = 1
        if w1n in w2n.hyponyms():
            w1[1] = 1
    
    if w1n:
        if w2n in w1n.hypernyms():
            w2[0] = 1
        if w2n in w1n.hyponyms():
            w2[1] = 1
            
    if (w1n and w1n in wn.synsets(word2)) or (w2n and w2n in wn.synsets(word1)):
        w1[2], w2[2] = 1, 1
    
    return w1, w2

In [44]:
wn.synset('blue.n.01').hypernyms()

[Synset('chromatic_color.n.01')]

In [47]:
wn.synsets('chromatic_color')

[Synset('chromatic_color.n.01')]